# Code Generator for Unit Tests and Comments/Docstrings

## Setup

In [1]:
import os

from dotenv import load_dotenv

from openai import OpenAI
import anthropic

from huggingface_hub import login
from transformers import AutoTokenizer, TextStreamer, AutoModelForCausalLM

import gradio as gr

In [2]:
load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('CLAUDE_API_KEY')
hf_token = os.getenv('HF_TOKEN')

In [3]:
openai = OpenAI()
claude = anthropic.Anthropic(api_key = anthropic_api_key)

In [4]:
OPENAI = 'o3-mini-2025-01-31'
CLAUDE = 'claude-3-5-sonnet-20240620'

QWEN = 'Qwen/CodeQwen1.5-7B-Chat'

In [5]:
TESTING = 'Unit Tests'
COMMENTING = 'Docstrings/Comments'

## System and User Prompt for Unit Test and Comments

In [6]:
system_message_comment = """ You are an AI programming documentation assisstant. Your task is to generate clear, concise, 
and informativ docstrings for the provided code block given by the user. 
Analyze the code to understand its functionality and intent. Then produce a detailed documentation that includes:
- a short summary what the code does.
- a short description of the parameters, including their expected types
- a short explanation what the function returns 
- if it's a complex code, and only then, some key insights
- if applicable how the function can be used
Ensure your documentation is written in clear gramatically correct english and in standard concentions (e.g PEP 257 for Python). 
It should be understandable and maintainable for other developers """

In [7]:
system_message_tests = """ You are an AI assisstant specialized for creating unit tests. Your task is to gnerate high-quality
unit tests for code provided by the user.
First analyze the code and identify the main functionality, parameters, return values and possible edge cases.
Create comprehensive unit tests that cover the following aspects:
- normal use cases with expected inputs and outputs
- boundary cases and extreme values
- error handling and exceptions
- edge cases 
Use the appropriate testing framework for the programming language (e.g., pytest for Python, etc.) and explain to the user why you 
chose this specific framework.
Structure the tests clearly with meaningful test names and add comments to explain the test logic.
If the code block does not provide enough context, as for the necessary details.
Supplemenet your response with a brief explanation of the testing strategy and suggestions for improving test coverage. """


In [8]:
def user_prompt_comment(code):
    user_prompt = f"""Please add detailed docstrings to the following code: 
                    {code} """
    return user_prompt

In [9]:
def user_prompt_tests(code):
    user_prompt = f""" Please generate unit tests for the following code using the appropriate framework: 
                    {code} """
    return user_prompt

## Define Model Functions

In [10]:
def stream_gpt(system_message, user_prompt):
    stream = openai.chat.completions.create(
        model = OPENAI,
        messages = [
            {'role': 'system', 'content': system_message},
            {'role': 'user', 'content': user_prompt}
        ],
        stream = True
    )

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        yield response

In [11]:
def stream_claude(system_message, user_prompt):
    response = claude.messages.stream(
        model = CLAUDE,
        max_tokens = 2000,
        system = system_message, 
        messages = [
            {'role': 'user', 'content': user_prompt}
        ], 
        temperature = 0.4
    )
    reply = ""
    with response as stream:
        for text in stream.text_stream:
            reply += text or ""
            yield reply

In [12]:
def stream_qwen(system_message, user_prompt):
    tokenizer = AutoTokenizer.from_pretrained(QWEN)
    model = AutoModelForCausalLM.from_pretrained(QWEN, device_map = 'gpu')
    streamer = TextStreamer(tokenizer)
    inputs = tokenizer.apply_chat_template(
        conv = [
            {'role': 'system', 'content': system_message},
            {'role': 'user', 'content': user_prompt}
        ],
        tokenize = False,
        add_generation_prompt = True
    )

    stream = model.text_generation(
        prompt = inputs, 
        stream = True,
        details = True,
        max_new_tokens = 2000
    )
    reply = ""
    for text in stream: 
        reply += text.token.text or ""
        yield reply    

In [13]:
def define_prompts(code, operation):
    if operation == 'Unit Tests':
        system_message = system_message_tests
        user_prompt = user_prompt_tests(code)
    elif operation == 'Docstrings/Comments':
        system_message = system_message_comment
        user_prompt = user_prompt_comment(code)
    else: 
        return 'Unknown operation', ''

    return system_message, user_prompt

In [14]:
def create_test_comment(code, model, operation):
    
    system_message, user_prompt = define_prompts(code, operation)
    
    if model == 'GPT-o3-mini':
        gen = stream_gpt(system_message, user_prompt)
    elif model == 'Claude-3.5-sonnet':
        gen = stream_claude(system_message, user_prompt)
    elif model == 'CodeQwen':
        gen =  stream_qwen(system_message, user_prompt)
    else: 
        gen =  'Unknown Model'

    result = ''
    for text in gen:
        result = text
    return result

## Creating easy Gradio UI 

In [15]:
def create_ui():

    with gr.Blocks(title = 'Code Generator') as ui:
        gr.Markdown('# Code Generator for Unit Testing and Docstrings')
    
        with gr.Row():
            with gr.Column(min_width = 500):
                code = gr.Textbox(label = 'Enter your Code', 
                                  placeholder = 'Code...', lines = 20
                                 )
                model = gr.Dropdown(['GPT-o3-mini', 'Claude-3.5-sonnet', 'CodeQwen'],
                                    label = 'Choose your Model',
                                    value = 'GPT-o3-mini'
                                   )
                operation = gr.Dropdown(['Unit Tests', 'Docstrings/Comments'],
                                        label = 'Choose operation',
                                        value = 'Unit Tests'
                                       )
                generate_button = gr.Button('Generate')
    
            with gr.Column():
                output = gr.Textbox(label = 'Generated Output',
                                    lines = 20
                                   )
    
        generate_button.click(fn = create_test_comment, inputs = [code, model, operation],
                              outputs = output,
                             )
    return ui

In [16]:
ui = create_ui()
ui.launch(inbrowser = True)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
